In [1]:
import numpy as np
import os
import time
import cv2
import itertools
import os
import shutil
import GMM_updation_uni as GMMu
import pickle as pkl
import itertools
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from scipy import misc
from scipy.optimize import linear_sum_assignment as lsaa
from collections import Counter
from sklearn.metrics import v_measure_score as vms
from sklearn.metrics import v_measure_score as vms
from sklearn.metrics import adjusted_rand_score as ars
from sklearn.metrics import fowlkes_mallows_score as fms
from sklearn.metrics import adjusted_mutual_info_score as amis
from sklearn.metrics import homogeneity_score as hs
from sklearn.metrics import completeness_score as cs

In [2]:
cntl = 0 

def overlap_in_percent(ref, new):
	lft = max(new[0]-new[2]/2, ref[0]-ref[2]/2)
	rt = min(new[0]+new[2]/2, ref[0]+ref[2]/2)
	top = max(new[1]-new[3]/2, ref[1]-ref[3]/2)
	bot = min(new[1]+new[3]/2, ref[1]+ref[3]/2)
	if rt-lft < 0 or bot-top < 0:
		return 0.0
	else:
		return 100.0 * (rt-lft)*(bot-top) / (0.0001 + max(new[2]*new[3], ref[2]*ref[3]))

def euc_dist_sq(x1,x2):
	return np.sum((x1-x2)**2)

def accuracy_score(y_act, y_pred):
	Cluster_Matrix = np.zeros((max(y_act)+1, max(y_pred)+1))
	print "Cluster_Matrix shape:"
	for i in range(len(y_act)):
		Cluster_Matrix[y_act[i], y_pred[i]]+=1.0	
	Inv_Cluster_Matrix = -1*Cluster_Matrix
	row, col = lsaa(Inv_Cluster_Matrix)
	accu = np.sum(Cluster_Matrix[row, col])/float(np.sum(np.sum(Cluster_Matrix)))
	print(Cluster_Matrix.astype(int))
	print(row)
	print(col)
	return accu

def clustering_score(ls_actual, ls_pred):

	dc = {0:0,1:1,2:2,6:3,9:4,11:5}
	for j in range(len(ls_actual)):
		ls_actual[j] = dc[ls_actual[j]]

	vm_score = vms(ls_actual, ls_pred)
	c_score = cs(ls_actual, ls_pred)
	h_score = hs(ls_actual, ls_pred)
	fm_score = fms(ls_actual, ls_pred)
	accu_score = accuracy_score(ls_actual, ls_pred)

	return vm_score, c_score, h_score, fm_score, accu_score

def update_labels(y_ls,np_t):
	ls_pred = []
	tu = np.unique(np_t)
	ls_tracks = []
	for i in range(len(tu)):
		ls_tracks.append([])

	dt = dict()
	for i in range(len(tu)):
		dt[tu[i]] = i

	print("y_ls: " + str(len(y_ls)))
	print("np_t: " + str(len(np_t)))

	for i in range(len(np_t)):
		tid = np_t[i]
		ls_tracks[dt[tid]].append(y_ls[i])

	dic_id = dict()

	for i in range(len(ls_tracks)):
		t = ls_tracks[i]
		c = Counter(t)
		value, count = c.most_common()[0]
		dic_id[tu[i]]=value

	for i in range(len(np_t)):
		tid = np_t[i]
		ls_pred.append(dic_id[tid]) 

	return ls_pred

def get_avg_feats(X_tr, np_lt, size=20):
	dt = dict()
	lb = dict()
	print("X_tr shape: "+str(X_tr.shape))

	for i in range(X_tr.shape[0]):
		tid = np_lt[i,0]
		if tid in dt:
			dt[tid].append(X_tr[i,:])
			lb[tid] = np_lt[i,1]
		else:
			dt[tid] = [X_tr[i,:]]
			lb[tid] = np_lt[i,1]
	ls_X = []
	ls_y = []
	ls_len = []
	for i in dt:
		fid = lb[i]
		X_t = dt[i]
		ls_ff = []
		for j in range(len(X_t)):
			ls_ff.append(X_t[j])
			if (j+1)%size==0:
				#print(len(ls_ff))
				#print(ls_ff)
				ls_X.append(np.mean(np.array(ls_ff),axis=0))
				ls_y.append(fid)
				ls_len.append(len(ls_ff))
				ls_ff = []
		if(len(ls_ff)>0):
			ls_X.append(np.mean(np.array(ls_ff),axis=0))
			ls_y.append(fid)
			ls_len.append(len(ls_ff))
			ls_ff = []
	np_X = np.array(ls_X)
	np_y = np.array(ls_y)
	np_l = np.array(ls_len)
	print(np_X.shape)
	print(np_y.shape)
	print(np_l.shape)
	print(sum(np_l))
	return np_X, np_y, np_l

In [3]:
def get_feats_and_cluster(np_lt, ls_feats_list, type="kmeans"):

	X_tr = np.array(ls_feats_list)
	print(X_tr.shape)
	y_tr = np_lt[:,1]
	tid = np_lt[:,0]
	print(y_tr.shape)
	print(tid.shape)
	# exit()
	n_comp = 6
	
	if(type=='gmm'):
		modelcurr = GaussianMixture(n_components=n_comp, covariance_type='tied', tol=0.001, 
			reg_covar=1e-06, max_iter=100, n_init=1, init_params='kmeans', 
			weights_init=None, means_init=None, precisions_init=None, 
			random_state=None, warm_start=False, verbose=0, verbose_interval=10)
		modelcurr.fit(X_tr)
		y = modelcurr.predict(X_tr)
		y_ls = []
		y_ = []
		for i in range(y.shape[0]):
			y_ls.append(y[i])
		for i in range(y_tr.shape[0]):
			y_.append(y_tr[i])
		y_pred = update_labels(y_ls, tid)
	elif (type=='kmeans'):
		model = KMeans(n_clusters=n_comp)
		y = model.fit_predict(X_tr)
		y_ls = []
		y_ = []
		for i in range(y.shape[0]):
			y_ls.append(y[i])
		for i in range(y_tr.shape[0]):
			y_.append(y_tr[i])
		y_pred = update_labels(y_ls, tid)
	elif (type=='sfc'):
		np_X, np_y, np_l = get_avg_feats(X_tr, np_lt, size=40)
		model = KMeans(n_clusters=n_comp)
		y = model.fit_predict(np_X)
		y_pred = []
		y_ = []
		for i in range(np_y.shape[0]):
			for j in range(np_l[i]):
				y_.append(np_y[i])
				y_pred.append(y[i])

	vm_score, c_score, h_score, fm_score, accu_score = clustering_score(y_, y_pred)
	return vm_score, c_score, h_score, fm_score, accu_score, n_comp

In [6]:
with open("./resources/ls_feats_list_buffy.pkl", 'rb' ) as fp:
    ls_feats_list = pkl.load(fp)
np_lt = np.load("./resources/buffy_tl.npy")

In [7]:
#######################K-Means##############################

lsa = []
lsv = []
lsc = []
lscm = []
lsh = []
lsf = []
lsac = []
for k in range(10):
    vm_score, c_score, h_score, fm_score, accu, ncl = get_feats_and_cluster(np_lt, ls_feats_list, "kmeans")
    print("KMeans Accu: " + str(accu))
    lsh.append(h_score)
    lscm.append(c_score)
    lsv.append(vm_score)
    lsc.append(ncl)
    lsf.append(fm_score)
    lsa.append(accu)
np_h = np.array(lsh)
np_cm = np.array(lscm)
np_v = np.array(lsv)
np_c = np.array(lsc)
np_f = np.array(lsf)
np_a = np.array(lsa)
print "Homo: " + str(np.mean(np_h)) + "+-" + str(np.std(np_h))
print "Cmpl: " + str(np.mean(np_cm)) + "+-" + str(np.std(np_cm))

print "VM score: " + str(np.mean(np_v)) + "+-" + str(np.std(np_v))
print "FM score: " + str(np.mean(np_f)) + "+-" + str(np.std(np_f))
print "Accu: " + str(np.mean(np_a)) + "+-" + str(np.std(np_a))
print "No of clusters: " + str(np.mean(np_c)) + "+-" + str(np.std(np_c))

(17337, 128)
(17337,)
(17337,)
y_ls: 17337
np_t: 17337
Cluster_Matrix shape:
[[   0    0    0    0  139  815]
 [4024   59 1126    0    0   57]
 [ 102    0    0 2400   90    0]
 [   0 2917   59  180  161  119]
 [   0    0    0    0    0  963]
 [  38    0    0    0 4088    0]]
[0 1 2 3 4 5]
[2 0 3 1 5 4]
KMeans Accu: 0.830132087443
(17337, 128)
(17337,)
(17337,)
y_ls: 17337
np_t: 17337
Cluster_Matrix shape:
[[   0  139    0  815    0    0]
 [   0    0   59   57 1126 4024]
 [2400   90    0    0    0  102]
 [ 180  192 2886  119   59    0]
 [   0    0    0  963    0    0]
 [   0 4088    0    0    0   38]]
[0 1 2 3 4 5]
[4 5 0 2 3 1]
KMeans Accu: 0.828344004153
(17337, 128)
(17337,)
(17337,)
y_ls: 17337
np_t: 17337
Cluster_Matrix shape:
[[ 815    0    0    0    0  139]
 [  57 4024   59 1126    0    0]
 [   0  102    0    0 2400   90]
 [ 119    0 2917   59  180  161]
 [ 963    0    0    0    0    0]
 [   0   38    0    0    0 4088]]
[0 1 2 3 4 5]
[3 1 4 2 0 5]
KMeans Accu: 0.830132087443
(173

In [8]:
lsa = []
lsv = []
lsc = []
lscm = []
lsh = []
lsf = []
lsac = []
for k in range(10):
    vm_score, c_score, h_score, fm_score, accu, ncl = get_feats_and_cluster(np_lt, ls_feats_list, "gmm")
    print("GMM Accu: " + str(accu))
    lsh.append(h_score)
    lscm.append(c_score)
    lsv.append(vm_score)
    lsc.append(ncl)
    lsf.append(fm_score)
    lsa.append(accu)
np_h = np.array(lsh)
np_cm = np.array(lscm)
np_v = np.array(lsv)
np_c = np.array(lsc)
np_f = np.array(lsf)
np_a = np.array(lsa)
print "Homo: " + str(np.mean(np_h)) + "+-" + str(np.std(np_h))
print "Cmpl: " + str(np.mean(np_cm)) + "+-" + str(np.std(np_cm))

print "VM score: " + str(np.mean(np_v)) + "+-" + str(np.std(np_v))
print "FM score: " + str(np.mean(np_f)) + "+-" + str(np.std(np_f))
print "Accu: " + str(np.mean(np_a)) + "+-" + str(np.std(np_a))
print "No of clusters: " + str(np.mean(np_c)) + "+-" + str(np.std(np_c))


(17337, 128)
(17337,)
(17337,)
y_ls: 17337
np_t: 17337
Cluster_Matrix shape:
[[   0  139    0    0  815    0]
 [4118    0 1091    0   57    0]
 [ 229    0    0 2363    0    0]
 [  63   72  178  180    0 2943]
 [   0    0    0    0  963    0]
 [  38 4088    0    0    0    0]]
[0 1 2 3 4 5]
[2 0 3 5 4 1]
GMM Accu: 0.834919536252
(17337, 128)
(17337,)
(17337,)
y_ls: 17337
np_t: 17337
Cluster_Matrix shape:
[[ 139  815    0    0    0    0]
 [   0  935 4272   59    0    0]
 [   0    0  229    0 2363    0]
 [  72  178   63 1131    0 1992]
 [   0  963    0    0    0    0]
 [4088    0   38    0    0    0]]
[0 1 2 3 4 5]
[3 2 4 5 1 0]
GMM Accu: 0.788948491665
(17337, 128)
(17337,)
(17337,)
y_ls: 17337
np_t: 17337
Cluster_Matrix shape:
[[ 139    0    0    0  313  502]
 [   0 4083    0 1091    0   92]
 [   0 2592    0    0    0    0]
 [  72  483 2703    0    0  178]
 [   0    0    0    0    0  963]
 [4088   38    0    0    0    0]]
[0 1 2 3 4 5]
[4 1 3 2 5 0]
GMM Accu: 0.700813289496
(17337, 128)


In [10]:
#######################Track-based##############################

lsa = []
lsv = []
lsc = []
lscm = []
lsh = []
lsf = []
lsac = []
for k in range(10):
    vm_score, c_score, h_score, fm_score, accu, ncl = get_feats_and_cluster(np_lt, ls_feats_list, "sfc")
    print("Track-based Accu: " + str(accu))
    lsh.append(h_score)
    lscm.append(c_score)
    lsv.append(vm_score)
    lsc.append(ncl)
    lsf.append(fm_score)
    lsa.append(accu)
np_h = np.array(lsh)
np_cm = np.array(lscm)
np_v = np.array(lsv)
np_c = np.array(lsc)
np_f = np.array(lsf)
np_a = np.array(lsa)
print "Homo: " + str(np.mean(np_h)) + "+-" + str(np.std(np_h))
print "Cmpl: " + str(np.mean(np_cm)) + "+-" + str(np.std(np_cm))

print "VM score: " + str(np.mean(np_v)) + "+-" + str(np.std(np_v))
print "FM score: " + str(np.mean(np_f)) + "+-" + str(np.std(np_f))
print "Accu: " + str(np.mean(np_a)) + "+-" + str(np.std(np_a))
print "No of clusters: " + str(np.mean(np_c)) + "+-" + str(np.std(np_c))

(17337, 128)
(17337,)
(17337,)
X_tr shape: (17337, 128)
(532, 128)
(532,)
(532,)
17337
Cluster_Matrix shape:
[[   0    0   99    0    0  855]
 [4183    0    0   96    0  987]
 [ 141    0   50    0 2401    0]
 [  40   40  111 2963  183   99]
 [   0  963    0    0    0    0]
 [  38    0 4070    0    0   18]]
[0 1 2 3 4 5]
[5 0 4 3 1 2]
Track-based Accu: 0.890292438138
(17337, 128)
(17337,)
(17337,)
X_tr shape: (17337, 128)
(532, 128)
(532,)
(532,)
17337
Cluster_Matrix shape:
[[  99    0    0    0    0  855]
 [   0    0    0 4339   96  831]
 [  50 2401    0  141    0    0]
 [ 111  183   40   59 2963   80]
 [   0    0  963    0    0    0]
 [4108    0    0    0    0   18]]
[0 1 2 3 4 5]
[5 3 1 4 2 0]
Track-based Accu: 0.901482378728
(17337, 128)
(17337,)
(17337,)
X_tr shape: (17337, 128)
(532, 128)
(532,)
(532,)
17337
Cluster_Matrix shape:
[[ 311    0    0    0   99  544]
 [  57    0   96 4183    0  930]
 [   0 2401    0  141   50    0]
 [   0  183 2884   40  151  178]
 [ 963    0    0    0